Результат в этом проекте получен с помощью модели RandomForestRegressor с добавлением данных по средней стоимости за метр с учетом количества комнат и района.

## Загружаем файлы и библиотеки:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('max_rows', 50)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data_test = pd.read_csv('test.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [5]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [6]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [7]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


## Исправляем выбросы:

In [8]:
def prepare_rooms_data(df):
    df.loc[df['Rooms']>5,['Rooms']]=2
    df.loc[df['Rooms']<1,['Rooms']]=2
    return df

In [9]:
data = prepare_rooms_data(data)
data_test = prepare_rooms_data(data_test)

In [10]:
def prepare_square_data(df):
    df.loc[df['Square']<15,['Square']]=df['Square'].mean()
    df.loc[df['Square']>250,['Square']]=df['Square'].mean()
    return df

In [11]:
data = prepare_square_data(data)
data_test = prepare_square_data(data_test)

In [12]:
def prepare_lsquare_data(df):
    df.loc[df['LifeSquare']>250,['LifeSquare']]=df['LifeSquare'].mean()
    df['LifeSquare']=df['LifeSquare'].fillna(0)
    df.loc[df['LifeSquare']<1,['LifeSquare']]=df['LifeSquare'].mean()
    return df

In [13]:
data = prepare_lsquare_data(data)
data_test = prepare_lsquare_data(data_test)

In [14]:
def prepare_ksquare_data(df,source_df):
    df.loc[df['Square']<df['KitchenSquare'],['KitchenSquare']]=8
    df.loc[df['KitchenSquare']<1,'KitchenSquare']=source_df['KitchenSquare'].mean()
    df.loc[df['KitchenSquare']>40,'KitchenSquare']=source_df['KitchenSquare'].mean()
    return df

In [15]:
data = prepare_ksquare_data(data,data)
data_test = prepare_ksquare_data(data_test,data)

In [16]:
data_test.loc[data_test['Floor']>70,['Floor']]=18

In [17]:
data.loc[data['HouseYear']==20052011,'HouseYear']=2008
data.loc[data['HouseYear']==4968,'HouseYear']=1968

In [18]:
def prepare_obj_data(df):
    df['Shops_2'] = (df['Shops_2'] == 'A').astype(int)
    df['Ecology_2'] = (df['Ecology_2'] == 'A').astype(int)
    df['Ecology_3'] = (df['Ecology_3'] == 'A').astype(int)
    return df

In [19]:
data = prepare_obj_data(data)
data_test = prepare_obj_data(data_test)

In [20]:
def prepare_health_data(df):
    df['Healthcare_1']=df['Healthcare_1'].fillna(0)
    return df

In [21]:
data = prepare_health_data(data)
data_test = prepare_health_data(data_test)

In [22]:
data_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.906200,56.551621,34.549276,6.130931,8.620000,12.601000,1984.392600,0.119874,0.009600,0.029800,24.933800,5406.900000,8.262600,601.536400,1.31940,4.242800,0.082400
std,4832.674037,44.179466,0.805935,18.954485,15.676042,3.573496,5.396353,6.789213,18.573149,0.120070,0.097518,0.170052,17.532202,4026.614773,23.863762,948.929796,1.47994,4.777365,0.275001
min,1.000000,0.000000,1.000000,16.319015,1.181139,1.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,4221.750000,21.000000,1.000000,41.951045,26.061113,5.000000,4.000000,9.000000,1973.000000,0.019509,0.000000,0.000000,6.000000,1564.000000,0.000000,0.000000,0.00000,1.000000,0.000000
50%,8320.500000,37.000000,2.000000,53.051069,29.118838,6.135233,7.000000,12.000000,1977.000000,0.072158,0.000000,0.000000,25.000000,5285.000000,2.000000,30.000000,1.00000,3.000000,0.000000
75%,12598.250000,77.000000,2.000000,66.285129,41.760597,9.000000,12.000000,17.000000,2000.000000,0.195781,0.000000,0.000000,36.000000,7287.000000,5.000000,990.000000,2.00000,6.000000,0.000000
max,16795.000000,212.000000,5.000000,223.453689,169.901701,40.000000,46.000000,99.000000,2020.000000,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000,1.000000


## Добавляем столбец со средней ценой за метр по районам с учетом количества комнат:

In [23]:
data['Price_m'] = data['Price'] / data['Square']

In [24]:
price=data['Price'].mean()
price_mean_by_rooms=data.groupby(['Rooms'],as_index=False).agg({'Price_m':'mean'}).\
                    rename(columns={'Price_m':'mean_price_by_rooms'})
price_mean_by_distr_rooms=data.groupby(['DistrictId','Rooms'],as_index=False).agg({'Price_m':'mean'}).\
                    rename(columns={'Price_m':'mean_price_dr'})

In [25]:
price_mean_by_distr_rooms.head()

,DistrictId,Rooms,mean_price_dr
0,0,1.0,3743.512621
1,0,2.0,3631.502943
2,0,3.0,3774.993359
3,1,1.0,3383.905490
4,1,2.0,3159.351031


In [26]:
price_mean_by_rooms

,Rooms,mean_price_by_rooms
0,1.0,3940.059213
1,2.0,3886.241440
2,3.0,3813.916275
3,4.0,3998.205057
4,5.0,4002.615580


In [27]:
def add_mean_price(df,price_mean_by_distr_rooms,price_mean_by_rooms,price):
    df=pd.merge(df,price_mean_by_distr_rooms,on=['DistrictId','Rooms'],how='left')
    df=pd.merge(df,price_mean_by_rooms,on='Rooms',how='left') 
    df['mean_price_dr']=df['mean_price_dr'].fillna(df['mean_price_by_rooms'])
    df['mean_price_dr']=df['mean_price_dr'].fillna(price)
    df['mean_price_by_rooms']= df['mean_price_by_rooms'].fillna(price)
    return df

In [28]:
data = add_mean_price(data,price_mean_by_distr_rooms,price_mean_by_rooms,price)
data_test = add_mean_price(data_test,price_mean_by_distr_rooms,price_mean_by_rooms,price)

## Делим на тренировочную и валидационную выборки:

In [29]:
train,valid=train_test_split(data,test_size=0.3, random_state=42)

In [30]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 6252 to 4705
Data columns (total 23 columns):
Id                     3000 non-null int64
DistrictId             3000 non-null int64
Rooms                  3000 non-null float64
Square                 3000 non-null float64
LifeSquare             3000 non-null float64
KitchenSquare          3000 non-null float64
Floor                  3000 non-null int64
HouseFloor             3000 non-null float64
HouseYear              3000 non-null int64
Ecology_1              3000 non-null float64
Ecology_2              3000 non-null int32
Ecology_3              3000 non-null int32
Social_1               3000 non-null int64
Social_2               3000 non-null int64
Social_3               3000 non-null int64
Healthcare_1           3000 non-null float64
Helthcare_2            3000 non-null int64
Shops_1                3000 non-null int64
Shops_2                3000 non-null int32
Price                  3000 non-null float64
Price_m       

In [31]:
feats=['mean_price_dr','Square','LifeSquare','KitchenSquare','Floor','HouseFloor',
       'HouseYear','Ecology_1','Ecology_2','Ecology_3','Social_1','Social_3','Shops_2','Helthcare_2']

In [32]:
model=RF(n_estimators=200,max_depth=13,max_features=5,random_state=42)

In [33]:
model.fit(train.loc[:,feats],train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [34]:
y_pred=model.predict(valid.loc[:,feats])

In [35]:
r2(valid['Price'],y_pred)

0.7712356125659793

In [36]:
r2(train['Price'],model.predict(train.loc[:,feats]))

0.9352630720272396

In [37]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 21 columns):
Id                     5000 non-null int64
DistrictId             5000 non-null int64
Rooms                  5000 non-null float64
Square                 5000 non-null float64
LifeSquare             5000 non-null float64
KitchenSquare          5000 non-null float64
Floor                  5000 non-null int64
HouseFloor             5000 non-null float64
HouseYear              5000 non-null int64
Ecology_1              5000 non-null float64
Ecology_2              5000 non-null int32
Ecology_3              5000 non-null int32
Social_1               5000 non-null int64
Social_2               5000 non-null int64
Social_3               5000 non-null int64
Healthcare_1           5000 non-null float64
Helthcare_2            5000 non-null int64
Shops_1                5000 non-null int64
Shops_2                5000 non-null int32
mean_price_dr          5000 non-null float64
mean_price_by_roo

In [38]:
data_test['Price']=model.predict(data_test.loc[:,feats])

In [40]:
data_test.loc[:,['Id','Price']].to_csv('YDavydyuk_predictions.csv',index=None)